## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,80.64,62,100,3.00,overcast clouds
1,1,Zhigansk,RU,66.7697,123.3711,53.98,64,100,14.14,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,41.49,83,49,5.06,scattered clouds
3,3,Monteiro,BR,-7.8894,-37.1200,78.60,55,67,12.44,broken clouds
4,4,Anito,PH,12.4497,125.2886,79.05,87,79,3.60,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,80.64,62,100,3.00,overcast clouds
3,3,Monteiro,BR,-7.8894,-37.1200,78.60,55,67,12.44,broken clouds
4,4,Anito,PH,12.4497,125.2886,79.05,87,79,3.60,broken clouds
6,6,Itoman,JP,26.1247,127.6694,84.40,89,100,11.50,overcast clouds
13,13,Sterling,US,39.0062,-77.4286,86.23,70,75,5.99,broken clouds
15,15,Hithadhoo,MV,-0.6000,73.0833,83.39,77,36,15.01,scattered clouds
18,18,Stephenville,US,32.2207,-98.2023,89.87,51,0,5.01,clear sky
19,19,Lorengau,PG,-2.0226,147.2712,81.30,80,7,9.64,clear sky
27,27,Hilo,US,19.7297,-155.0900,79.29,100,100,1.99,moderate rain
34,34,Atuona,PF,-9.8000,-139.0333,76.41,78,24,13.18,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
updated_df = preferred_cities_df.dropna()
updated_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = updated_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,80.64,overcast clouds,42.6001,-73.9662,
3,Monteiro,BR,78.60,broken clouds,-7.8894,-37.1200,
4,Anito,PH,79.05,broken clouds,12.4497,125.2886,
6,Itoman,JP,84.40,overcast clouds,26.1247,127.6694,
13,Sterling,US,86.23,broken clouds,39.0062,-77.4286,
15,Hithadhoo,MV,83.39,scattered clouds,-0.6000,73.0833,
18,Stephenville,US,89.87,clear sky,32.2207,-98.2023,
19,Lorengau,PG,81.30,clear sky,-2.0226,147.2712,
27,Hilo,US,79.29,moderate rain,19.7297,-155.0900,
34,Atuona,PF,76.41,light rain,-9.8000,-139.0333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
updated_hotel_df = hotel_df.dropna()
updated_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Monteiro,BR,78.60,broken clouds,-7.8894,-37.1200,Pousada Seu Tião
4,Anito,PH,79.05,broken clouds,12.4497,125.2886,El Felix Cottages
6,Itoman,JP,84.40,overcast clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
13,Sterling,US,86.23,broken clouds,39.0062,-77.4286,Hampton Inn & Suites Washington-Dulles Interna...
15,Hithadhoo,MV,83.39,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [13]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
updated_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in updated_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = updated_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = updated_hotel_df[["Lat", "Lng"]]
max_temp = updated_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))